In [2]:
import json
import pandas as pd
import zipfile
from transformers import AutoTokenizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re

/home/hice1/akumar978/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hice1/akumar978/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [1]:
import os

# Dynamically set scratch folder to the current working directory
scratch_folder = os.path.join(os.getcwd(), "cache")  # Create a 'cache' folder in your current working directory

# Set environment variables for caching
os.environ["TRANSFORMERS_CACHE"] = os.path.join(scratch_folder, "transformers")
os.environ["TORCH_HOME"] = os.path.join(scratch_folder, "torch")
os.environ["HF_HOME"] = os.path.join(scratch_folder, "huggingface")

# Ensure the cache directories exist
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)
os.makedirs(os.environ["TORCH_HOME"], exist_ok=True)
os.makedirs(os.environ["HF_HOME"], exist_ok=True)

# # Example usage with transformers
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "RohitSuresh15/llama3.2_1b-medical-v1"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name, 
#     cache_dir=os.environ["TRANSFORMERS_CACHE"], 
#     force_download=True
# )
# tokenizer = AutoTokenizer.from_pretrained(
#     model_name, 
#     cache_dir=os.environ["TRANSFORMERS_CACHE"], 
#     force_download=True
# )
print(f"Transformers cache directory: {os.environ['TRANSFORMERS_CACHE']}")
print(f"Torch cache directory: {os.environ['TORCH_HOME']}")
print(f"HuggingFace cache directory: {os.environ['HF_HOME']}")

Transformers cache directory: /storage/ice1/4/0/akumar978/BDA Project/cache/transformers
Torch cache directory: /storage/ice1/4/0/akumar978/BDA Project/cache/torch
HuggingFace cache directory: /storage/ice1/4/0/akumar978/BDA Project/cache/huggingface


In [3]:
with open('release_evidences.json', 'r') as f:
    evidence_dict = json.load(f)

with open('release_conditions.json', 'r') as f:
    condition_dict = json.load(f)
    
with open('sample_train_data_200k.json', 'r') as f:
    train_data = json.load(f)

with open('sample_test_data_2000.json', 'r') as f:
    test_data = json.load(f)
    
with open('sampled_validate_combined_data_200k.json','r') as f:
    val_data = json.load(f)

In [4]:
print(train_data[0])

{'instruction': 'Provide Diagnosis', 'input': 'Patient age is 67, sex is M.  Antecedents: Have you had one or several flare ups of chronic obstructive pulmonary disease (COPD) in the past year? Y ; Do you have a chronic obstructive pulmonary disease (COPD)? Y ; Do you work in construction? Y ; Do you work in the mining sector? Y ; Have you traveled out of the country in the last 4 weeks? N . Symptoms: Have you noticed a wheezing sound when you exhale? Y ; Do you have a cough that produces colored or more abundant sputum than usual? Y ; Do you have a cough? Y ; Have you noticed a wheezing sound when you exhale? Y . ', 'output': ' Differential diagnosis is: Bronchitis, Acute COPD exacerbation / infection, Pneumonia, Bronchospasm / acute asthma exacerbation, Bronchiectasis and the Disease can be Acute COPD exacerbation / infection '}


In [5]:
def load_patients(file_path):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        with zip_ref.open(zip_ref.namelist()[0]) as f:
            return pd.read_csv(f)

In [15]:
train_patients = load_patients('release_train_patients.zip')
val_patients = load_patients('release_validate_patients.zip')
test_patients = load_patients('release_test_patients.zip')

In [7]:
train_patient=train_patients[:6000]

In [8]:
val_patient=val_patients[:1200]

In [9]:
test_patient=test_patients[:2000]

In [10]:
def create_text_representation(row, output_path):
    # Gather patient information
    age = row['AGE']
    sex = row['SEX']
    pathology = row['PATHOLOGY']
    initial_evidence = row['INITIAL_EVIDENCE']
    evidences = eval(row['EVIDENCES'])
    evidences = [initial_evidence] + evidences
    # For differential diagnosis 
    # data = eval(row['DIFFERENTIAL_DIAGNOSIS'])
    # diseases = [item[0] for item in data]
    # diseases = ', '.join(diseases)

    description = f"Age: {age}, Sex: {sex}. "
    # Add detailed symptoms and antecedents
    symptom_texts = []
    antecedents = []
    for evidence_code in evidences:
        # Separate multi-choice evidence by value
        if "_@_" in evidence_code:
            evidence, value = evidence_code.split('_@_')
            evidence_text = evidence_dict[evidence]['question_en']
            value_text = evidence_dict[evidence]['value_meaning'].get(value)
            value_text = value_text['en'] if value_text is not None else value
            if value_text=='N':
                value_text = 'No'
            if value_text =='Y':
                value_text = 'Yes'
            if value_text == 'NA':
                value_text = 'Not Applicable'
                
            if evidence_dict[evidence]['is_antecedent']:
                antecedents.append(f"{evidence_text}: {value_text}")
            else:
                symptom_texts.append(f"{evidence_text}: {value_text}")
        else:
            if evidence_dict[evidence_code]['is_antecedent']:
                antecedents.append(evidence_dict[evidence_code]['question_en']+'Yes')
            else:
                symptom_texts.append(evidence_dict[evidence_code]['question_en']+'Yes')

    description += "History:" + "; ".join(antecedents) + ". Symptoms: " + "; ".join(symptom_texts) + "."
    label = pathology
    
#     with open(output_path, 'a', encoding='utf-8') as f:
#         chat_format = {
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": str(description)
#                 },
#                 {
#                     "role": "assistant",
#                     "content": str(label)
#                 }
#             ]
#         }
#         # Write each JSON object on a new line without pretty printing
#         json.dump(chat_format, f, ensure_ascii=False)
#         f.write('\n')  # Add newline after each JSON object

In [20]:
test_results = test_patients.apply(create_text_representation,output_path ='test_result.csv', axis=1)

In [21]:
test_results.head()

0    None
1    None
2    None
3    None
4    None
dtype: object

In [13]:
#train_results = pd.DataFrame(train_patients.apply(create_text_representation, axis=1).toList(), columns=['text', 'label'])
val_patient = val_patient
val_result = val_patient.apply(create_text_representation, output_path ='val_result.jsonl', axis=1 )
#val_results = pd.DataFrame(val_results.tolist(), columns=['text', 'label'])

test_patient = test_patient
test_result = test_patient.apply(create_text_representation,output_path ='test_result.jsonl', axis=1)
#test_results = pd.DataFrame(test_results.tolist(), columns=['text', 'label'])

train_patient = train_patient
train_result = train_patient.apply(create_text_representation,output_path ='train_result.jsonl', axis=1)

In [14]:
val_result.to_csv('val_patients_with_text.csv', index=False)
test_result.to_csv('test_patients_with_text.csv', index=False)
train_result.to_csv('train_patients_with_text.csv', index=False)

In [15]:
all_labels = pd.concat([train_patients['PATHOLOGY'], val_patients['PATHOLOGY'], test_patients['PATHOLOGY']])

# Fit Label Encoder
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Transform pathologies to numerical labels
train_patients_label = label_encoder.transform(train_patients['PATHOLOGY'])
val_patients_label = label_encoder.transform(val_patients['PATHOLOGY'])
test_patients_label = label_encoder.transform(test_patients['PATHOLOGY'])

In [33]:
# Load and display the first few entries
def inspect_dataset(file_path, num_entries=5):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            print(f"Showing first {num_entries} entries of the dataset:\n")
            for i, line in enumerate(f):
                if i >= num_entries:
                    break
                # Parse the JSON object from each line
                chat_format = json.loads(line.strip())
                print(json.dumps(chat_format, indent=4, ensure_ascii=False))
                print("-" * 80)  # Separator for clarity
    except Exception as e:
        print(f"Error while reading the dataset: {e}")

# Inspect the dataset
inspect_dataset("test_result.jsonl")

Showing first 5 entries of the dataset:

{
    "messages": [
        {
            "role": "user",
            "content": "Age: 49, Sex: F. History:Are you significantly overweight compared to people of the same height as you?Yes; Do you drink alcohol excessively or do you have an addiction to alcohol?Yes; Do you have a hiatal hernia?Yes; Do you think you are pregnant or are you currently pregnant?Yes; Have you traveled out of the country in the last 4 weeks?: No. Symptoms: Do you have a cough?Yes; Do you have pain somewhere, related to your reason for consulting?Yes; Characterize your pain:: haunting; Characterize your pain:: sensitive; Characterize your pain:: tugging; Characterize your pain:: burning; Do you feel pain somewhere?: lower chest; Do you feel pain somewhere?: upper chest; Do you feel pain somewhere?: hypochondrium(R); How intense is the pain?: 6; Does the pain radiate to another location?: lower chest; Does the pain radiate to another location?: upper chest; How precisel

In [12]:
import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-11-21 14:27:56.507614: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 14:27:56.538945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732217276.571467  262564 cuda_dnn.cc:8321] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732217276.580513  262564 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 14:27:56.598506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [13]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.097 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 9.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.dev941. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
from datasets import Dataset
# Load your dataset (assuming it is in JSONL format)
data = []
with open("train_result.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

# Convert to Hugging Face Dataset format
hf_dataset = Dataset.from_list(data)

In [12]:
from unsloth.chat_templates import get_chat_template
from datasets import Dataset

# Initialize the chat template with your tokenizer
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3",  # Specify your chat template
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    }
)

# Define the formatting function
def formatting_prompts_func(examples):
    convos = examples["messages"]  # Replace with your column name
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

# Apply the formatting function to your dataset
formatted_dataset = hf_dataset.map(formatting_prompts_func, batched=True)

Map: 100%|██████████| 6000/6000 [00:00<00:00, 30629.70 examples/s]


In [13]:
print(formatted_dataset[0]["text"])

<|user|>
Age: 18, Sex: M. History:Do you live with 4 or more people?Yes; Do you smoke cigarettes?Yes; Have you traveled out of the country in the last 4 weeks?: No; Are you exposed to secondhand cigarette smoke on a daily basis?Yes. Symptoms: Do you have a fever (either felt or measured with a thermometer)?Yes; Have you had significantly increased sweating?Yes; Do you have pain somewhere, related to your reason for consulting?Yes; Characterize your pain:: sensitive; Characterize your pain:: heavy; Do you feel pain somewhere?: forehead; Do you feel pain somewhere?: cheek(R); Do you feel pain somewhere?: temple(L); How intense is the pain?: 4; Does the pain radiate to another location?: nowhere; How precisely is the pain located?: 3; How fast did the pain appear?: 3; Do you have a cough that produces colored or more abundant sputum than usual?Yes; Do you have a fever (either felt or measured with a thermometer)?Yes; Do you have a sore throat?Yes; Do you have a cough?Yes.<|end|>
<|assista

In [76]:
dataset = load_dataset("json", data_files={"train": "sample_train_data_200k.json", "validation": "sampled_validate_combined_data_200k.json", "test": "sample_test_data_2000.json"})

Generating train split: 200026 examples [00:02, 97763.19 examples/s]
Generating validation split: 9062 examples [00:00, 100838.85 examples/s]
Generating test split: 290 examples [00:00, 38557.92 examples/s]


In [77]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.097 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 9.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.dev941. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [13]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [79]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "o_proj","gate_proj","k_proj", "v_proj","up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = True,
    random_state = 3411,
    max_seq_length = 1024,
    use_rslora = False,  # Rank stabilized LoRA
    loftq_config = None, # LoftQ
)

In [8]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    instruction = 'Perform Diagnosis'
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for inputx, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, inputx, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

NameError: name 'dataset' is not defined

In [81]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset['train'],
    #eval_dataset = dataset['validation'],
    dataset_text_field = "text",
    max_seq_length = 1024,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        num_train_epochs = 1,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "gemma_outputs",
        lr_scheduler_type = "linear",
        optim = "adamw_8bit",
        seed = 3411,
    ),
)
trainer.train()

Map: 100%|██████████| 200026/200026 [02:09<00:00, 1543.88 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 200,026 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 3,125
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss
1,2.090000
2,2.067200
3,2.060200
4,2.129800
5,2.095300
6,2.110300
7,2.127800
8,2.072800
9,2.038700
10,2.081600


TrainOutput(global_step=3125, training_loss=0.1467162138557434, metrics={'train_runtime': 9897.5748, 'train_samples_per_second': 20.21, 'train_steps_per_second': 0.316, 'total_flos': 3.133169055106007e+18, 'train_loss': 0.1467162138557434, 'epoch': 0.9998400255959047})

In [1]:
model.save_pretrained("phi_lora_model")
model.save_pretrained_merged("phi_merged_model", tokenizer, save_method = "merged_16bit",)

NameError: name 'model' is not defined

In [9]:
def generate_text(model,tokenizer, text, max_length):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
def load_unsloth_model_and_tokenizer(model_path):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_path,
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
        local_files_only = True,
    )
    return model, tokenizer

In [66]:
def generate_predictions(test_file, model, tokenizer, output_file, max_new_tokens=200):
    """
    Generate predictions for test samples and save to a JSONL file.

    Args:
        test_file (str): Path to the test samples JSONL file.
        model: Loaded Unsloth model.
        tokenizer: Tokenizer for the model.
        output_file (str): Path to save the model predictions JSONL file.
        max_new_tokens (int): Maximum number of new tokens to generate.
    """
    predictions = []
    
    with open(test_file, 'r') as f:
        for line in f:
            try:
                item = json.loads(line)
                instruction = item.get("instruction", "Perform Diagnosis")
                human_input = item["input"]

                # Format input for the model using the Alpaca-style template
                prompt = alpaca_prompt.format(instruction, human_input, "")

                # Tokenize the input
                inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

                # Generate predictions
                outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
                prediction_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

                # Save the prediction
                predictions.append({
                    "instruction": instruction,
                    "input": human_input,
                    "output": prediction_text.strip()
                })
            except json.JSONDecodeError:
                print("Skipping invalid line")
                continue

    # Write all predictions to the output file in JSONL format
    with open(output_file, 'w') as f:
        for prediction in predictions:
            f.write(json.dumps(prediction) + '\n')

    print(f"Predictions saved to {output_file}")


In [11]:
model_path = "gemma_outputs/checkpoint-3125"
test_file = "sample_test_data_2000.json"  # Test samples JSONL file
model_output_file = "model_predictions.json"  # Output predictions JSONL file
#model, tokenizer = load_unsloth_model_and_tokenizer(model_path)

In [68]:
FastLanguageModel.for_inference(model)
# Generate predictions
generate_predictions(test_file, model, tokenizer, model_output_file)
print(f"Predictions saved to {model_output_file}")

Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping invalid line
Skipping i

In [48]:
# model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "phi_lora_model", # YOUR MODEL YOU USED FOR TRAINING
#         max_seq_length = max_seq_length,
#         dtype = dtype,
#         load_in_4bit = load_in_4bit,
#     )
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# messages = [
#     {"from": "human", "value": "Age: 70, Sex: F. History:Have you been in contact with a person with similar symptoms in the past 2 weeks?Yes; Have you traveled out of the country in the last 4 weeks?: No. Symptoms: Do you have a cough?Yes; Have you had significantly increased sweating?Yes; Do you have pain somewhere, related to your reason for consulting?Yes; Characterize your pain:: sensitive; Characterize your pain:: heavy; Do you feel pain somewhere?: top of the head; Do you feel pain somewhere?: forehead; Do you feel pain somewhere?: cheek(R); Do you feel pain somewhere?: cheek(L); Do you feel pain somewhere?: occiput; How intense is the pain?: 7; Does the pain radiate to another location?: nowhere; How precisely is the pain located?: 3; How fast did the pain appear?: 0; Do you have a fever (either felt or measured with a thermometer)?Yes; Do you have a sore throat?Yes; Do you have diffuse (widespread) muscle pain?Yes; Do you have nasal congestion or a clear runny nose?Yes; Do you have a cough?Yes."},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     return_tensors = "pt",
# ).to("cuda")

In [69]:
with open(test_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            item = json.loads(line)
        except json.JSONDecodeError as e:
            print(f"Error on line: {line} -> {e}")

Error on line: [{"instruction": "Provide Diagnosis", "input": "Patient age is 59, sex is F.  Antecedents: Do you have severe Chronic Obstructive Pulmonary Disease (COPD)? Y ; Have you had one or several flare ups of chronic obstructive pulmonary disease (COPD) in the past year? Y ; Do you smoke cigarettes? Y ; Do you have a chronic obstructive pulmonary disease (COPD)? Y ; Have you ever been diagnosed with gastroesophageal reflux? Y ; Have you traveled out of the country in the last 4 weeks? N . Symptoms: Are you experiencing shortness of breath or difficulty breathing in a significant way? Y ; Are you experiencing shortness of breath or difficulty breathing in a significant way? Y ; Do you have a cough that produces colored or more abundant sputum than usual? Y ; Do you have a cough? Y . ", "output": " Differential diagnosis is: Bronchitis, Bronchiectasis, Pneumonia, Pulmonary neoplasm, Acute COPD exacerbation / infection, Guillain-Barre syndrome, Bronchospasm / acute asthma exacerbat

In [49]:
# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# # _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

In [21]:
FastLanguageModel.for_inference(model)
# Generate predictions
inp = "Age: 70, Sex: F. History:Have you been in contact with a person with similar symptoms in the past 2 weeks?Yes; Have you traveled out of the country in the last 4 weeks?: No. Symptoms: Do you have a cough?Yes; Have you had significantly increased sweating?Yes; Do you have pain somewhere, related to your reason for consulting?Yes; Characterize your pain:: sensitive; Characterize your pain:: heavy; Do you feel pain somewhere?: top of the head; Do you feel pain somewhere?: forehead; Do you feel pain somewhere?: cheek(R); Do you feel pain somewhere?: cheek(L); Do you feel pain somewhere?: occiput; How intense is the pain?: 7; Does the pain radiate to another location?: nowhere; How precisely is the pain located?: 3; How fast did the pain appear?: 0; Do you have a fever (either felt or measured with a thermometer)?Yes; Do you have a sore throat?Yes; Do you have diffuse (widespread) muscle pain?Yes; Do you have nasal congestion or a clear runny nose?Yes; Do you have a cough?Yes."
inputs = tokenizer(
[
    alpaca_prompt.format(
        "List your diagnosis (no explanation) in the form: Differential diagnosis is: Acute COPD exacerbation / infection, Bronchitis, Pneumonia, Tuberculosis, Acute pulmonary edema, Anaphylaxis, Guillain-Barre syndrome, Atrial fibrillation, Myocarditis, Pulmonary embolism, Bronchiectasis, Acute dystonic reactions, Myasthenia gravis, Anemia, Scombroid food poisoning, PSVT, SLE, Possible NSTEMI / STEMI, Chagas and the Disease can be Acute COPD exacerbation / infection ", # instruction
        inp, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nList your diagnosis (no explanation) in the form: Differential diagnosis is: Acute COPD exacerbation / infection, Bronchitis, Pneumonia, Tuberculosis, Acute pulmonary edema, Anaphylaxis, Guillain-Barre syndrome, Atrial fibrillation, Myocarditis, Pulmonary embolism, Bronchiectasis, Acute dystonic reactions, Myasthenia gravis, Anemia, Scombroid food poisoning, PSVT, SLE, Possible NSTEMI / STEMI, Chagas and the Disease can be Acute COPD exacerbation / infection \n\n### Input:\nAge: 70, Sex: F. History:Have you been in contact with a person with similar symptoms in the past 2 weeks?Yes; Have you traveled out of the country in the last 4 weeks?: No. Symptoms: Do you have a cough?Yes; Have you had significantly increased sweating?Yes; Do you have pain somewhere, related to your reason for consulting?Yes; Character